In [1]:
import pandas as pd
import numpy as np

In [2]:
address_list_fname = 'address_list.csv'
postcode_reference_fname = 'postcode_reference.csv'

In [3]:
def show_details(df):
    seperator = '\n\n'
    print(17*"=", "Info", 17*"=")
    print(df.info(), seperator)
    print(15*"=", "Describe", 15*"=")
    print(df.describe(), seperator)
    print(17*"=", "Head", 17*"=")
    return df.head()

In [4]:
address_list = pd.read_csv(address_list_fname)

show_details(address_list)

================= Info =================
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53027 entries, 0 to 53026
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   urn                53027 non-null  int64  
 1   Registration Date  53027 non-null  object 
 2   Latitude           53027 non-null  object 
 3   Longitude          53027 non-null  float64
 4   Location           53027 non-null  object 
 5   Unnamed: 5         3 non-null      object 
dtypes: float64(1), int64(1), object(4)
memory usage: 2.4+ MB
None 


=============== Describe ===============
                urn     Longitude
count  53027.000000  53027.000000
mean   26514.000000      0.069033
std    15307.720699      0.388743
min        1.000000      0.012110
25%    13257.500000      0.036439
50%    26514.000000      0.067826
75%    39770.500000      0.088708
max    53027.000000     51.606400 


================= Head =================


,urn,Registration Date,Latitude,Longitude,Location,Unnamed: 5
0,1,24/04/2012 00:00,51.6155,0.032659,"30, Monkhams Lane, Woodford Green, IG8 0NS",NaN
1,2,24/04/2012 00:00,51.5717,0.052316,"74, Canterbury Avenue, Ilford, IG1 3NG",NaN
2,3,24/04/2012 00:00,51.5988,0.013589,"80, Chelmsford Road, London, E18 2PP",NaN
3,4,24/04/2012 00:00,51.5958,0.074697,"236, Fullwell Avenue, Ilford, IG5 0XP",NaN
4,5,23/04/2012 00:00,51.5836,0.047408,"49, Torquay Gardens, Ilford, IG4 5PU",NaN


In [5]:
postcode_reference = pd.read_csv(postcode_reference_fname)

show_details(postcode_reference)

================= Info =================
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7732 entries, 0 to 7731
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   postcode             7732 non-null   object 
 1   postcode_introduced  7732 non-null   int64  
 2   postcode_terminated  3141 non-null   float64
 3   lat                  7732 non-null   float64
 4   long                 7732 non-null   float64
dtypes: float64(3), int64(1), object(1)
memory usage: 302.2+ KB
None 


=============== Describe ===============
       postcode_introduced  postcode_terminated          lat         long
count          7732.000000          3141.000000  7732.000000  7732.000000
mean         199046.260476        200438.812480    51.579223     0.066551
std            1207.007045           686.319147     0.020004     0.029568
min          198001.000000        198112.000000    51.543915     0.012515
25%          198001.

,postcode,postcode_introduced,postcode_terminated,lat,long
0,E11 1PA,198001,200007.0,51.584393,0.026455
1,E11 1PB,198001,NaN,51.583693,0.024807
2,E11 1PD,198001,NaN,51.584817,0.025362
3,E11 1PE,198001,NaN,51.582346,0.020660
4,E11 1PF,198001,NaN,51.583964,0.022767


### First comments on data quality
Very few NaNs in the data, the only columns are "Unnamed: 5" in `address_list` and "postcode_terminated" in `postcode_reference`.

I will look at "Unnamed: 5" to see what's given in there, and also "Registration Date" as that was not automatically converted to a datetime, and "Latitude" as that was not converted to float like "Longitude. I don't think NaNs in "postcode_terminated" are an issue, as of course not all postcodes are terminated.

In [6]:
address_list[address_list["Unnamed: 5"].notnull()]

,urn,Registration Date,Latitude,Longitude,Location,Unnamed: 5
5364,5365,01/09/2010 00:00,Stage 55 Decision Despatch date present,51.6064,http://planning.redbridge.gov.uk/swiftlg/apas/...,Works to TPO Tree(s)
8368,8369,27/10/2009 00:00,Stage 55 Decision Despatch date present,51.5988,http://planning.redbridge.gov.uk/swiftlg/apas/...,Works to Trees in Con. Area
25516,25517,09/03/2005 00:00,Stage 55 Decision Despatch date present,51.5641,http://planning.redbridge.gov.uk/swiftlg/apas/...,Full Planning Permission


In [7]:
# Look at Latitude

address_list["temp_convert_latitude"] = pd.to_numeric(address_list["Latitude"], errors="coerce")

address_list[address_list["temp_convert_latitude"].isnull()]

# The same 3 rows which were populated in "Unnamed: 5" are the ones with the invalid Latitude.

,urn,Registration Date,Latitude,Longitude,Location,Unnamed: 5,temp_convert_latitude
5364,5365,01/09/2010 00:00,Stage 55 Decision Despatch date present,51.6064,http://planning.redbridge.gov.uk/swiftlg/apas/...,Works to TPO Tree(s),NaN
8368,8369,27/10/2009 00:00,Stage 55 Decision Despatch date present,51.5988,http://planning.redbridge.gov.uk/swiftlg/apas/...,Works to Trees in Con. Area,NaN
25516,25517,09/03/2005 00:00,Stage 55 Decision Despatch date present,51.5641,http://planning.redbridge.gov.uk/swiftlg/apas/...,Full Planning Permission,NaN


In [8]:
# Drop the temporary column

address_list.drop(columns=["temp_convert_latitude"], inplace=True)

# Convert the Latitudes to floats, coercing errors into NaNs
address_list['Latitude'] = pd.to_numeric(address_list['Latitude'], errors='coerce')

In [9]:
# Look at registration date

address_list["temp_convert_reg_date"] = pd.to_datetime(address_list["Registration Date"], errors="coerce")

address_list[address_list["temp_convert_reg_date"].isnull()]

,urn,Registration Date,Latitude,Longitude,Location,Unnamed: 5,temp_convert_reg_date


In [10]:
# No issues with converting registration date, so I will set the column values to be temp ones, 
# and drop the temp column

address_list["Registration Date"] = address_list["temp_convert_reg_date"]

address_list.drop(columns=["temp_convert_reg_date"], inplace=True)

address_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53027 entries, 0 to 53026
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   urn                53027 non-null  int64         
 1   Registration Date  53027 non-null  datetime64[ns]
 2   Latitude           53024 non-null  float64       
 3   Longitude          53027 non-null  float64       
 4   Location           53027 non-null  object        
 5   Unnamed: 5         3 non-null      object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 2.4+ MB


In [11]:
# URN is in fact a indentifier, this may prove useful later

address_list[address_list['urn'].duplicated(keep=False)]

,urn,Registration Date,Latitude,Longitude,Location,Unnamed: 5


### Summary at current stage

What I think I need is a static reference on the data types that I need, e.g. a dictionary with the column names as the keys, and the values are the functions to do conversion, e.g. pd.to_numeric.

So a function to read in and do this type conversion on the columns of interest, which at this point look to be

* `address_list`: "Location", "Longitude", "Latitude", "Registration Date"
  
*  `postcode_reference`: "postcode", "lat", "long", "postcode_introduced", "postcode_terminated"

I should also create "Location Postcode" column, where I've parsed the postcode in the "Location" column for comparison to what I've brought in from the `postcode_reference` data. This column will eventually be deleted.

Given the task also involves adding a validated column, I believe this process should retain all original data. So if there is a missing longitude or latitude, then I should mark the column invalid, in line with what was decribed in the task for Location:

> ...otherwise False if the postcode is not the same or if the Location column does not have a postcode.


### Next steps

Checkout postcode "postcode_introduced" and "postcode_terminated" in `postcode_reference`. 

At a glance they look to be dates formatted as **YYYYMM**, I expect the "Registration Date" will lie between only one "postcode_introduced" and "postcode_terminated" pair, and that that pair is the valid one. 


I believe that if a match is found with a terminated postcode, that it is still valid as we may be dealing with historical records in `address_list` whose data integrity needs to be checked retroactively, for example as part of some reconcilliation purpose. 

Whilst it is possible that valid actually carries the implication of current, the brief does not state this explicity, so this is something I would try to clarify with the people who wrote it up, but this exercise is also about what choices do you make and why in the face of ambiguity so I will stick with my initial assumption.

In [12]:
postcode_reference[['lat', 'long', 'postcode']][postcode_reference.duplicated(['lat', 'long'], keep=False)]

,lat,long,postcode
48,51.584474,0.027527,E11 1UX
49,51.584474,0.027527,E11 1WD
50,51.584474,0.027527,E11 1WE
51,51.584474,0.027527,E11 1WF
52,51.584474,0.027527,E11 1WJ
...,...,...,...
7632,51.575581,0.123033,RM6 4ZE
7633,51.575581,0.123033,RM6 4ZQ
7634,51.572334,0.120601,RM6 4ZR
7724,51.564735,0.124605,RM8 1XP


In [13]:
postcode_reference_slice = postcode_reference[['lat', 'long', 'postcode']][postcode_reference.duplicated(['lat', 'long'], keep=False)]
postcode_reference_slice = postcode_reference_slice.groupby(['lat', 'long']).nunique()

# These long, lat have multiple postcodes
postcode_reference_slice[postcode_reference_slice['postcode'] == 1]

,,postcode
lat,long,


In [14]:
postcode_reference_slice = postcode_reference[['postcode', 'postcode_terminated']][postcode_reference.duplicated(['lat', 'long'], keep=False)]

mask = postcode_reference_slice["postcode_terminated"].isna()
postcode_reference_slice = postcode_reference_slice[mask].groupby(['postcode']).nunique(dropna=False)

# Each postcode has only one NaN value in postcode_terminated, i.e. one current value
postcode_reference_slice[postcode_reference_slice['postcode_terminated'] != 1]

,postcode_terminated
postcode,


In [15]:
postcode_reference_slice = postcode_reference[['lat', 'long', 'postcode']][postcode_reference.duplicated(['lat', 'long'], keep=False)]
postcode_reference_slice['long_lat'] = postcode_reference_slice['long'].apply(str) + postcode_reference_slice['lat'].apply(str)
postcode_reference_slice = postcode_reference_slice.groupby(['postcode']).nunique()

# Each postcode has only one corresponding long lat
mask = (postcode_reference_slice['long_lat'] != 1)
postcode_reference_slice[mask]

,lat,long,long_lat
postcode,,,


In [16]:
postcode_reference_slice = postcode_reference[['lat', 'long', 'postcode_terminated']][postcode_reference.duplicated(['lat', 'long'], keep=False)]

mask = postcode_reference_slice["postcode_terminated"].isna()
postcode_reference_slice = postcode_reference_slice[mask].groupby(['lat', 'long']).nunique(dropna=False)

# Each long and lat has only one NaN value in postcode_terminated, i.e. one current value
postcode_reference_slice[postcode_reference_slice['postcode_terminated'] != 1]

,,postcode_terminated
lat,long,


In [17]:
# It will be easier to do date comparisons with the strings converted 
# to datetime

import datetime

def custom_str_to_datetime(dt_str, dt_str_format='%Y%m'):
    try:
        if pd.isna(dt_str):
            return pd.NaT
        elif isinstance(dt_str, pd.Timestamp):
            return dt_str
        else:
            dt_str = str(dt_str)
            return datetime.datetime.strptime(dt_str, dt_str_format)
    except ValueError:
        return pd.NaT
    
postcode_reference["postcode_introduced"] = postcode_reference["postcode_introduced"].apply(custom_str_to_datetime)
postcode_reference["postcode_terminated"] = postcode_reference["postcode_terminated"].apply(custom_str_to_datetime)

postcode_reference.head()

,postcode,postcode_introduced,postcode_terminated,lat,long
0,E11 1PA,1980-01-01,NaT,51.584393,0.026455
1,E11 1PB,1980-01-01,NaT,51.583693,0.024807
2,E11 1PD,1980-01-01,NaT,51.584817,0.025362
3,E11 1PE,1980-01-01,NaT,51.582346,0.020660
4,E11 1PF,1980-01-01,NaT,51.583964,0.022767


To find the postcode, I looked around at how to get a valid postcode
First was to simply to split the location string and get the last value. However there was the potential that it wouldn't be the last value, if present at all.

Looking around, I found a discussion on StackOverflow discussing regex for UK postcodes, https://stackoverflow.com/a/51885364, which looked reasonable, so I will take the regex approach with the the shortened but not simpflified regex.

In [18]:
import re


example = address_list.iloc[12, :]
print(example)

#postcode_regex = r"^([A-Z][A-HJ-Y]?\d[A-Z\d]? ?\d[A-Z]{2}|GIR ?0A{2})$"
postcode_regex = "^(([Gg][Ii][Rr] 0[Aa]{2})|((([A-Za-z][0-9]{1,2})|(([A-Za-z][A-Ha-hJ-Yj-y][0-9]{1,2})|(([A-Za-z][0-9][A-Za-z])|([A-Za-z][A-Ha-hJ-Yj-y][0-9][A-Za-z]?))))\s?[0-9][A-Za-z]{2}))$"
postcodes = re.findall(postcode_regex, example['Location'])

if len(postcodes) == 0:
    print('No postcode found in', example['Location'])
elif len(postcodes) > 1:
    print('Multiple postcodes found in', example['Location'])
else:
    postcode = postcodes[0]

    print(postcode)

    lookup_mask = (postcode_reference['long'] == example['Longitude']) & (postcode_reference['lat'] == example['Latitude'])

    postcode_reference[(postcode_reference['long'] >= 0.0326) & (postcode_reference['long'] <= 0.0327)]

    postcode_reference[postcode_reference['postcode'] == postcode]

urn                                                13
Registration Date                 2012-04-19 00:00:00
Latitude                                      51.5813
Longitude                                     0.03219
Location             9, Warwick Road, London, E11 2DZ
Unnamed: 5                                        NaN
Name: 12, dtype: object
No postcode found in 9, Warwick Road, London, E11 2DZ


In [19]:
postcode_reference[postcode_reference['postcode'] == 'E11 2DZ']

,postcode,postcode_introduced,postcode_terminated,lat,long
113,E11 2DZ,1980-01-01,NaT,51.581294,0.033058


This issue is getting complex, I will stick with checking the end, and it can be an improvement to make later down the line to do regex matching

In [20]:
# If the last value is not a postcode, then the lookup will 
# yield nothing, in line with the expected behaviour of marking invalid
# for non existant postcodes

i = 0

def get_postcode(location):
    if ',' in location:
        postcode =  location.split(',')[-1].strip()   
        if ' ' not in postcode:
            global i
            i += 1
        return postcode
    return location
    
print(i, 'postcodes without space inbetween')

address_list['Postcode'] = address_list['Location'].apply(get_postcode)
address_list.head()

0 postcodes without space inbetween


,urn,Registration Date,Latitude,Longitude,Location,Unnamed: 5,Postcode
0,1,2012-04-24,51.6155,0.032659,"30, Monkhams Lane, Woodford Green, IG8 0NS",NaN,IG8 0NS
1,2,2012-04-24,51.5717,0.052316,"74, Canterbury Avenue, Ilford, IG1 3NG",NaN,IG1 3NG
2,3,2012-04-24,51.5988,0.013589,"80, Chelmsford Road, London, E18 2PP",NaN,E18 2PP
3,4,2012-04-24,51.5958,0.074697,"236, Fullwell Avenue, Ilford, IG5 0XP",NaN,IG5 0XP
4,5,2012-04-23,51.5836,0.047408,"49, Torquay Gardens, Ilford, IG4 5PU",NaN,IG4 5PU


Looking at the longitude and latitudes, it seems there are no exact matches, so I will need to define some sort of fuzzy matching.

Given we're looking for valid as matching postcode, my approach will be:

1. Parse postcode in address_list
2. Left join postcode_reference onto address_list on postcode
 * If they're a mismatch, I am marking as invalid anyway
3. Make a bool column on whether the registration date lies between the postcode dates
4. Make a bool column, calculating a distance between the longitude and latitudes from both Dataframes, and testing if it's below some threshold. 
5. Filter on the bool columns as a mask.
6. Check for duplicates, but I think there will be none as the postcode dates look to be correctly offset*.
7. Left join this filtered set back onto address list to provide the values of Invalid and Postcode, with defaults of False and NaN if not provided.

In [21]:
from geopy.distance import geodesic

# (lat, long)
example_coord = (51.5813, 0.03219)
other_coord = (51.581294, 0.033058)

geodesic(example_coord, other_coord).meters

60.170772908444654

In [22]:
postcode_reference

df = pd.merge(address_list, postcode_reference, 
         how='left', 
         left_on=['Longitude', 'Latitude'], 
         right_on = ['long','lat'])

df[df['postcode'].notnull()].head()

,urn,Registration Date,Latitude,Longitude,Location,Unnamed: 5,Postcode,postcode,postcode_introduced,postcode_terminated,lat,long


In [23]:
merged = address_list.merge(postcode_reference, how='left', left_on='Postcode', right_on='postcode')

def check_date_within_range(record):
    date = record['Registration Date']
    start = record['postcode_introduced']
    stop = record['postcode_terminated']
    
    if pd.isna(stop):
        return date >= start
    else:
        return stop > date >= start
    
merged['within_dates'] = merged.apply(check_date_within_range, axis=1)


def longitudes_and_latitudes_close(record, tol_meters=1):
    lat1 = record['Latitude']
    long1 = record['Longitude']
    lat2 = record['lat']
    long2 = record['long']
    
    return geodesic((lat1, long1), (lat2, long2)).meters <= tol_meters

merged['coordinates_close'] = merged.apply(check_date_within_range, axis=1)

matched = merged[merged['within_dates'] & merged['coordinates_close']]
matched = matched[['urn', 'Postcode']]
matched['validated'] = True

print(matched.info())

matched[matched.duplicated(keep=False)]

address_list.drop(columns=['Postcode'], inplace=True)

address_list = address_list.merge(matched, on='urn', how='left')

def replace_nans(validated):
    if pd.isna(validated):
        return False
    return validated

address_list['validated'] = address_list['validated'].apply(replace_nans)

address_list.head()

print(address_list.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24512 entries, 0 to 52873
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   urn        24512 non-null  int64 
 1   Postcode   24512 non-null  object
 2   validated  24512 non-null  bool  
dtypes: bool(1), int64(1), object(1)
memory usage: 598.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 53027 entries, 0 to 53026
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   urn                53027 non-null  int64         
 1   Registration Date  53027 non-null  datetime64[ns]
 2   Latitude           53024 non-null  float64       
 3   Longitude          53027 non-null  float64       
 4   Location           53027 non-null  object        
 5   Unnamed: 5         3 non-null      object        
 6   Postcode           24512 non-null  object        
 7   validated          5302